In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib
tf.config.list_physical_devices('GPU')

2022-09-05 17:36:22.301440: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-05 17:36:22.324404: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-05 17:36:22.324566: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
device_lib.list_local_devices()

2022-09-05 17:36:23.126703: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-05 17:36:23.128027: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-05 17:36:23.128222: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-05 17:36:23.128344: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2323034326551598700
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 4043374592
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10615807762163854015
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [3]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

/home/raph/anaconda3/envs/gpu/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Your labels map as a dictionary (zero is reserved):
label_map = {
    
    1: "Pedestrian",
    2: "People",
    3: "Bicycle",
    4: "Car",
    5: "Van",
    6: "Truck",
    7: "Tricycle",
    8: "Awning-tricycle",
    9: "Bus",
    10: "Motor",
    11: "Others",
    12: "None"
}
# file = open("Classeur1.txt","r")
# lignes = file.readlines()
# file.close()

# for ligne in lignes:
#     element = ligne.split(";")
#     cle = int(element[0])
#     print(cle)
#     data = element[1]
#     print(data)
#     label_map[cle] = data
print (label_map)

# If it's NOT split yet, specify the path to all images and annotations
images_in = "/home/raph/Bureau/VisDrone2019-DET-train/images_new"
annotations_in = "/home/raph/Bureau/VisDrone2019-DET-train/annotations_new"

{1: 'Pedestrian', 2: 'People', 3: 'Bicycle', 4: 'Car', 5: 'Van', 6: 'Truck', 7: 'Tricycle', 8: 'Awning-tricycle', 9: 'Bus', 10: 'Motor', 11: 'Others', 12: 'None'}


In [5]:
import os
import random
import shutil

def split_dataset(images_path, annotations_path, val_split, test_split, out_path):
  """Splits a directory of sorted images/annotations into training, validation, and test sets.

  Args:
    images_path: Path to the directory with your images (JPGs).
    annotations_path: Path to a directory with your VOC XML annotation files,
      with filenames corresponding to image filenames. This may be the same path
      used for images_path.
    val_split: Fraction of data to reserve for validation (float between 0 and 1).
    test_split: Fraction of data to reserve for test (float between 0 and 1).
  Returns:
    The paths for the split images/annotations (train_dir, val_dir, test_dir)
  """
  _, dirs, _ = next(os.walk(images_path))

  train_dir = os.path.join(out_path, 'train')
  val_dir = os.path.join(out_path, 'validation')
  test_dir = os.path.join(out_path, 'test')

  IMAGES_TRAIN_DIR = os.path.join(train_dir, 'images_new')
  IMAGES_VAL_DIR = os.path.join(val_dir, 'images_new')
  IMAGES_TEST_DIR = os.path.join(test_dir, 'images_new')
  os.makedirs(IMAGES_TRAIN_DIR, exist_ok=True)
  os.makedirs(IMAGES_VAL_DIR, exist_ok=True)
  os.makedirs(IMAGES_TEST_DIR, exist_ok=True)

  ANNOT_TRAIN_DIR = os.path.join(train_dir, 'annotations_new')
  ANNOT_VAL_DIR = os.path.join(val_dir, 'annotations_new')
  ANNOT_TEST_DIR = os.path.join(test_dir, 'annotations_new')
  os.makedirs(ANNOT_TRAIN_DIR, exist_ok=True)
  os.makedirs(ANNOT_VAL_DIR, exist_ok=True)
  os.makedirs(ANNOT_TEST_DIR, exist_ok=True)

  # Get all filenames for this dir, filtered by filetype
  filenames = os.listdir(os.path.join(images_path))
  filenames = [os.path.join(images_path, f) for f in filenames if (f.endswith('.jpg'))]
  # Shuffle the files, deterministically
  filenames.sort()
  random.seed(42)
  random.shuffle(filenames)
  # Get exact number of images for validation and test; the rest is for training
  val_count = int(len(filenames) * val_split)
  test_count = int(len(filenames) * test_split)
  for i, file in enumerate(filenames):
    source_dir, filename = os.path.split(file)
    annot_file = os.path.join(annotations_path, filename.replace("jpg", "xml"))
    if i < val_count:
      shutil.copy(file, IMAGES_VAL_DIR)
      shutil.copy(annot_file, ANNOT_VAL_DIR)
    elif i < val_count + test_count:
      shutil.copy(file, IMAGES_TEST_DIR)
      shutil.copy(annot_file, ANNOT_TEST_DIR)
    else:
      shutil.copy(file, IMAGES_TRAIN_DIR)
      shutil.copy(annot_file, ANNOT_TRAIN_DIR)
  return (train_dir, val_dir, test_dir)

In [6]:
# We need to instantiate a separate DataLoader for each split dataset
train_dir, val_dir, test_dir = split_dataset(images_in, annotations_in,
                                                 val_split=0.2, test_split=0.2,
                                                 out_path='split-dataset')
train_data = object_detector.DataLoader.from_pascal_voc(
    os.path.join(train_dir, 'images_new'),
    os.path.join(train_dir, 'annotations_new'), label_map=label_map)
validation_data = object_detector.DataLoader.from_pascal_voc(
    os.path.join(val_dir, 'images_new'),
    os.path.join(val_dir, 'annotations_new'), label_map=label_map)
test_data = object_detector.DataLoader.from_pascal_voc(
    os.path.join(test_dir, 'images_new'),
    os.path.join(test_dir, 'annotations_new'), label_map=label_map)
    
print(f'train count: {len(train_data)}')
print(f'validation count: {len(validation_data)}')
print(f'test count: {len(test_data)}')

train count: 3464
validation count: 1154
test count: 1154


In [9]:
#spec = object_detector.EfficientDetLite0Spec(hparams{'max_instances_per_image': MAXOBJECTCOUNT})
spec = object_detector.EfficientDetLite0Spec(
  model_name='efficientdet-lite3', 
  uri='https://tfhub.dev/tensorflow/efficientdet/lite3/feature-vector/1', 
  #hparams={'max_instances_per_image': 8000},
  strategy='gpus',
  model_dir = '/home/raph/Bureau/VisDrone2019-DET-train/VisDrone2019-DET-train/control_point' )

In [10]:
model = object_detector.create(train_data=train_data, 
                               model_spec=spec,  
                               epochs=5, 
                               batch_size=15,
                               train_whole_model=False)

Epoch 1/5


2022-09-05 17:42:34.941806: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.89GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-09-05 17:42:34.942869: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.89GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-09-05 17:42:34.942981: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 654.32MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-09-05 17:42:34.943008: W tensorflow/core/common_runtime/bfc_alloca

230/230 [==============================] - 116s 423ms/step - det_loss: 3.1573 - cls_loss: 1.6156 - box_loss: 0.0308 - reg_l2_loss: 0.0086 - loss: 3.1658 - learning_rate: 0.0134 - gradient_norm: 3.3251
Epoch 2/5
230/230 [==============================] - 99s 431ms/step - det_loss: 2.7184 - cls_loss: 1.3005 - box_loss: 0.0284 - reg_l2_loss: 0.0091 - loss: 2.7274 - learning_rate: 0.0129 - gradient_norm: 2.4771
Epoch 3/5
230/230 [==============================] - 100s 435ms/step - det_loss: 2.5709 - cls_loss: 1.2615 - box_loss: 0.0262 - reg_l2_loss: 0.0092 - loss: 2.5801 - learning_rate: 0.0059 - gradient_norm: 2.3950
Epoch 4/5
230/230 [==============================] - 101s 441ms/step - det_loss: 2.4909 - cls_loss: 1.2504 - box_loss: 0.0248 - reg_l2_loss: 0.0093 - loss: 2.5002 - learning_rate: 9.3644e-04 - gradient_norm: 2.3942
Epoch 5/5
230/230 [==============================] - 101s 440ms/step - det_loss: 2.4737 - cls_loss: 1.2355 - box_loss: 0.0248 - reg_l2_loss: 0.0093 - loss: 2.4830 

In [11]:
model.evaluate(test_data)

2022-09-05 17:51:37.687601: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.25GiB (rounded to 2415919104)requested by op keras_layer_1/StatefulPartitionedCall/StatefulPartitionedCall/StatefulPartitionedCall/efficientnet-lite3/StatefulPartitionedCall/blocks_1/conv2d/Conv2D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-09-05 17:51:37.687995: I tensorflow/core/common_runtime/bfc_allocator.cc:1027] BFCAllocator dump for GPU_0_bfc
2022-09-05 17:51:37.688038: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (256): 	Total Chunks: 311, Chunks in use: 310. 77.8KiB allocated for chunks. 77.5KiB in use in bin. 13.6KiB client-requested in use in bin.
2022-09-05 17:51:37.688064: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (512): 	Total Chunks:

ResourceExhaustedError: Graph execution error:

2 root error(s) found.
  (0) RESOURCE_EXHAUSTED:  OOM when allocating tensor with shape[64,144,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node blocks_1/conv2d/Conv2D}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[GatherNd_430/_2250]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

  (1) RESOURCE_EXHAUSTED:  OOM when allocating tensor with shape[64,144,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node blocks_1/conv2d/Conv2D}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor

In [12]:
TFLITE_FILENAME = 'efficientdet-lite3-raph.tflite'
LABELS_FILENAME = 'labels.txt'

In [13]:
model.export(export_dir='/home/raph/Bureau/VisDrone2019-DET-train/modele', tflite_filename=TFLITE_FILENAME, label_filename=LABELS_FILENAME,
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])

2022-09-05 17:52:27.807508: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2022-09-05 17:52:36.342532: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-09-05 17:52:36.342588: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-09-05 17:52:36.343608: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpxu6_ovvi
2022-09-05 17:52:36.442001: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2022-09-05 17:52:36.442041: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmpxu6_ovvi
2022-09-05 17:52:36.684526: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-09-05 17:52:

In [15]:
model.evaluate_tflite("/home/raph/Bureau/VisDrone2019-DET-train/modele/efficientdet-lite3-flo.tflite", test_data)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


1154/1154 [==============================] - 498s 431ms/step



{'AP': 9.217083e-08,
 'AP50': 1.5361805e-07,
 'AP75': 1.5361805e-07,
 'APs': 7.478315e-09,
 'APm': 7.6871044e-07,
 'APl': 0.0,
 'ARmax1': 0.0,
 'ARmax10': 0.00013368984,
 'ARmax100': 0.00015597147,
 'ARs': 5.7471265e-05,
 'ARm': 0.00028089888,
 'ARl': 0.0,
 'AP_/Pedestrian': 0.0,
 'AP_/People': 0.0,
 'AP_/Bicycle': 0.0,
 'AP_/Car': 1.1060499e-06,
 'AP_/Van': 0.0,
 'AP_/Truck': 0.0,
 'AP_/Tricycle': 0.0,
 'AP_/Awning-tricycle': 0.0,
 'AP_/Bus': 0.0,
 'AP_/Motor': 0.0,
 'AP_/Others': 0.0,
 'AP_/None': 0.0}